In [1]:
# filtering future warnings/ ignoring future warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
#importing required libraries and importing Cifar10 dataset
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D 
import os, time
import matplotlib.pyplot as plt
import numpy as np
from keras.constraints import maxnorm
from keras import optimizers
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
#num_predictions 

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

In [5]:
# developing CNN layers
model = Sequential()
#1st convolution set of 2 conv layers followed by max pooling has 32 filters each
model.add(Conv2D(32, (3, 3), activation='relu', name='set1_conv1', input_shape=x_train.shape[1:], padding= 'same'))
model.add(Conv2D(32, (3, 3), activation='relu',name='set1_conv2'))
model.add(MaxPooling2D(pool_size=(2, 2), name='set1_maxpool1'))
model.add(Dropout(0.25))

#2nd convolution set of 2 conv layers followed by max pooling has 64 filters each
model.add(Conv2D(64, (3, 3), padding='same',activation='relu', name='set2_conv3'))
model.add(Conv2D(64, (3, 3), activation='relu', name='set2_conv4'))
model.add(MaxPooling2D(pool_size=(2, 2), name='set2_maxpool2'))
model.add(Dropout(0.25))

#Adding fully connected layer
model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='relu', name='fully_connected_layer1'))
model.add(Dropout(0.5))
model.add(Dense((num_classes), activation='softmax', name='prediction_layer'))

# adding a optimizer
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd , metrics=['accuracy'])
#keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
set1_conv1 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
set1_conv2 (Conv2D)          (None, 30, 30, 32)        9248      
_________________________________________________________________
set1_maxpool1 (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
set2_conv3 (Conv2D)          (None, 15, 15, 64)        18496     
_________________________________________________________________
set2_conv4 (Conv2D)          (None, 13, 13, 64)        36928     
_________________________________________________________________
set2_maxpool2 (MaxPooling2D) (None, 6, 6, 64)          0         
__________

In [6]:
train_datagen = ImageDataGenerator(rotation_range=0, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.2, horizontal_flip=False, fill_mode='nearest',validation_split= 0, data_format=None, vertical_flip=False )
train_datagen.fit(x_train)

In [10]:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(x_test, y_test))

Epoch 1/100
1563/1563 [==============================] - 653s 418ms/step - loss: 2.3099 - acc: 0.1002 - val_loss: 2.3128 - val_acc: 0.1000
Epoch 2/100
1563/1563 [==============================] - 820s 525ms/step - loss: 2.3091 - acc: 0.0993 - val_loss: 2.3097 - val_acc: 0.1000
Epoch 3/100
1563/1563 [==============================] - 891s 570ms/step - loss: 2.3092 - acc: 0.1007 - val_loss: 2.3061 - val_acc: 0.1000
Epoch 4/100
1563/1563 [==============================] - 916s 586ms/step - loss: 2.3095 - acc: 0.0988 - val_loss: 2.3089 - val_acc: 0.1000
Epoch 5/100
1563/1563 [==============================] - 936s 599ms/step - loss: 2.3091 - acc: 0.1003 - val_loss: 2.3088 - val_acc: 0.1000
Epoch 6/100
1563/1563 [==============================] - 940s 602ms/step - loss: 2.3094 - acc: 0.0970 - val_loss: 2.3090 - val_acc: 0.1000
Epoch 7/100
1563/1563 [==============================] - 962s 616ms/step - loss: 2.3092 - acc: 0.1003 - val_loss: 2.3061 - val_acc: 0.1000
Epoch 8/100
1563/1563 [====

KeyboardInterrupt: 

In [11]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 30s 3ms/step
Test loss: 2.306232322311401
Test accuracy: 0.1


In [ ]:
plt.figure(0)
plt.plot(model.history['acc'],'r')
plt.plot(model.history['val_acc'],'g')
plt.xticks(np.arange(0, 101, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])